In [4]:
import os
import cv2
import numpy as np
from tqdm import tqdm

REBUILD_DATA = False

class DogVSCats():
    IMG_SIZE = 50
    CATS = "PetImages/Cat"
    DOGS = "PetImages/Dog"
    
    LABELS = {CATS : 0, DOGS: 1}
    training_data = []
    
    catCount = 0
    dogCount = 0
    
    def make_training_data(self):
        for label in tqdm(self.LABELS):
            for f in os.listdir(label):
                try:
                    path = os.path.join(label,f)
                    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                    img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE))
                    self.training_data.append([np.array(img), np.eye(2)[self.LABELS[label]]])

                    if label == self.CATS:
                        self.catCount += 1
                    elif label == self.DOGS:
                        self.dogCount += 1
                except Exception as e:
                    print(f)
                    
                    
        np.random.shuffle(self.training_data)
        np.save("training_data.npy", self.training_data)
        print("Cats: ", self.catCount)
        print("Dogs: ", self.dogCount)
        
if REBUILD_DATA:
    dogs_v_cats = DogVSCats()
    dogs_v_cats.make_training_data()

In [5]:
training_data = np.load("training_data.npy", allow_pickle=True)

FileNotFoundError: [Errno 2] No such file or directory: 'training_data.npy'

In [ ]:
print(len(training_data))

In [ ]:
print(training_data[0])

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(training_data[1][0], cmap = "gray")
plt.show()

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(1 ,   32, 5)
        self.conv2 = nn.Conv2d(32 ,  64, 5)
        self.conv3 = nn.Conv2d(64 , 128, 5)
        
        x = torch.randn(50,50).view(-1, 1 , 50 , 50)
        self._to_linear = None
        self.convs(x)
        
        self.fc1 = nn.Linear(self._to_linear, 512)
        self.fc2 = nn.Linear(512, 2)

    def convs(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2,2))
        
        if self._to_linear is None:
            s = x[0].shape
            self._to_linear = s[0] * s[1] * s[2]
            
        return x
    
    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return F.softmax(x, dim = 1)

In [ ]:
net = Net()

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(net.parameters(), lr = 0.001)

loss_function = nn.MSELoss()
X = torch.Tensor([i[0] for i in training_data]).view(-1,50,50)
X = X/255.0
y = torch.Tensor([i[1] for i in training_data])

VAL_PCT = 0.1
val_size = int(len(X)*VAL_PCT)
print(val_size)

In [ ]:
train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]

print(len(train_X) , len(test_X))

In [ ]:
BATCH_SIZE = 100
EPOCHS = 3

for epoch in range(EPOCHS):
    for i in tqdm(range(0,len(train_X) , BATCH_SIZE)):
        batch_X = train_X[i : i+BATCH_SIZE].view(-1,1,50,50)
        batch_y = train_y[i : i+BATCH_SIZE]
        
        optimizer.zero_grad()
        outputs = net(batch_X)
        loss = loss_function(outputs, batch_y)
        
        loss.backward()
        optimizer.step()
        
print(loss)

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for i in tqdm(range(len(test_X))):
        real_class = torch.argmax(test_y[i])
        net_out = net(test_X[i].view(-1,1,50,50))[0]
        predicted_class = torch.argmax(net_out)
        if predicted_class == real_class:
            correct += 1
        total += 1
        
print("Accuracy:", round(correct/total, 3))

In [ ]:
import matplotlib.pyplot as plt
img = training_data[199][0]
plt.imshow(img, cmap = "gray")
plt.show()
print(test_X[11].view(-1,1,50,50))
print(torch.tensor(img).view(-1,1,50,50))

img = (torch.tensor(img)/255).view(-1,1,50,50)

In [ ]:
net(img)

